In [1]:
from openai import OpenAI
OPENAI_KEY = open(".env", "r").read().strip()

client = OpenAI(api_key=OPENAI_KEY)

def message(role, content):
  return {
    "role": role,
    "content": content
  }

def chat(messages):
  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
  )
  response =  completion.choices[0].message.content
#   messages.append(message("assistant", response))
  return response


In [9]:
chat_msgs = []

In [5]:
output_struct = {"questions":["list of questions about the 3D model to be built, along with example options"]}

SYSTEM_PROMPT = f"""You're a 3D graphic designer & planner who specializes in creating 3D models schema based on descriptions of objects. Do not assume anything, you have to ask questions about the model, like how it has to be, its approx dimensions, texture, features and other user customizations too. return the response in this json format: {output_struct} only!"""

chat_msgs.append(message("system", SYSTEM_PROMPT))
chat_msgs.append(message("user", "I want a house"))

print(chat(chat_msgs))

{'questions': ['What are the approximate dimensions of the house (length, width, height)?', 'How many floors/stories should the house have (e.g., 1, 2, 3)?', 'What architectural style are you aiming for (e.g., modern, traditional, Victorian, colonial)?', 'How many rooms do you want in total, and what specific types are needed (e.g., 3 bedrooms, 2 bathrooms, kitchen, living room, dining room)?', 'Are there any special features you want to include (e.g., garage, balcony, basement, attic, porch, swimming pool)?', 'What materials and textures do you prefer for the exterior and interior of the house (e.g., brick, wood, stone, glass)?', 'What color palette are you considering for the exterior and interior walls?', 'Do you need detailed interior features like furniture, appliances, and decor?', 'Should the house design include surrounding landscapes like a garden, driveway, fence, or patio?', 'Are there any accessibility features that need to be incorporated (e.g., ramps, wide doorways)?', 'D

In [6]:
output_struct = {"steps":["list of instruction to draw the 3D model"]}

SYSTEM_PROMPT = f"""You're a 3D graphic designer & planner who specializes in creating 3D models schema based on descriptions of objects. Take your time, think and solve step by step, and generate a concise and straightforward list of steps to draw the 3D model (Not 2D) from the description, including proper dimensions for each component. return the response in this json format: {output_struct} only!"""

chat_msgs.append(message("system", SYSTEM_PROMPT))
chat_msgs.append(message("user", "I want a house"))

print(chat(chat_msgs))

{'steps': ['Determine the overall dimensions of the house (length, width, height). Example: 50 ft x 40 ft x 25 ft.',
            'Create the base structure/floor plan of the house (rectangular or desired shape).',
            'Extrude the base to the height of the house for the exterior walls.',
            'Subdivide the base for different rooms as per requirements (e.g., bedrooms, bathrooms, kitchen, living room). Example: 3 bedrooms of 12 ft x 10 ft each, 2 bathrooms of 8 ft x 6 ft each.',
            'Add the second floor if needed by extruding another level. Include staircases for access.',
            'Design and extrude the roof based on the preferred style (e.g., gable, hip, flat). Example: Gable roof with a rise of 8 ft.',
            'Add windows and doors in their respective positions on the walls, using the desired style and dimensions. Example: 3 ft x 4 ft windows and 7 ft x 3 ft doors.',
            'Include special features such as balconies, porches, or garages. Example

In [8]:
output_struct = {"description_of_components":["list of description of each component of the 3D model"]}

SYSTEM_PROMPT = f"""You're a 3D graphic designer & planner who specializes in creating 3D models schema based on descriptions of objects. Take your time, think and solve step by step, and give description of overall exterior structure and other independent/individual components, think this description as a first step of making and printing a model using 3D printer. return the response in this json format: {output_struct} only!"""

chat_msgs = message("system", SYSTEM_PROMPT)
chat_msgs.append(message("user", "I want a house"))

questions = chat(chat_msgs)
print(questions)


{'description_of_components': 'Exterior Structure: The house will have an overall rectangular shape with dimensions of 50 ft length, 40 ft width, and 25 ft height. The exterior walls will be textured with a brick material. The roof will be a gable style with an 8 ft rise, covered in dark composite shingles.\\n
First Floor: The first floor will include a living room (15 ft x 20 ft), kitchen (12 ft x 15 ft), dining room (10 ft x 12 ft), and one bathroom (8 ft x 6 ft). Flooring will be a wooden texture.\\n
Second Floor: The second floor will contain three bedrooms (each 12 ft x 10 ft) and one bathroom (8 ft x 6 ft). Flooring will be carpeted texture. A staircase will connect the first and second floors.\\n
Windows and Doors: Each floor will have multiple windows (3 ft x 4 ft), with a total of 10 windows around the house. Doors will be standardized at 7 ft height and 3 ft width, with one main entrance door and internal doors for each room.\\n
Garage: A 20 ft x 10 ft garage attached to the 

In [11]:
import json
import ast

In [13]:
from uagents import Agent, Context
Questionnaire_agent = Agent(name="questionnaire_agent", seed="questionnaire_agent_seed")

In [17]:
@Questionnaire_agent.on_event("startup")
async def Questionnaire_agent(ctx: Context):
	ctx.logger.info("Questionnaire agent is kicking off")
    
	output_struct = {"questions":["list of questions about the 3D model to be built, along with sample options"]}
    
	SYSTEM_PROMPT = f"""You're a 3D graphic designer & planner who specializes in creating 3D models schema based on descriptions of objects. Do not assume anything, you have to ask questions about the model, like how it has to be, its approx dimensions, texture, features and other user customizations too. return the response in this json format: {output_struct} only!"""

	chat_msgs = message("system", SYSTEM_PROMPT)
	chat_msgs.append(message("user", "I want a house"))


	try:
		questions = ast.literal_eval(chat(chat_msgs))
		print(questions)

		questions = [q for q in questions["questions"]]
	except Exception as e:
		ctx.logger.error(e)
		print(e)
		questions = []

	ctx.logger.info(chat(chat_msgs))

In [19]:
Questionnaire_agent.run()

RuntimeError: This event loop is already running

In [ ]:
#👤: user input ✅
#🤖: generate questions ✅
#🤖: ask questions, clear any ambiguity 
#👤: answer questions
#💻: store user input, questions, answers
#🤖: generate detailed description of components using the stored info
#🤖: create series of steps to create 3D model and its components
#🤖: generate 3D model vrml file, see how to write a vrml file properly
#💻: convert vrml file to stl file
#🎞️: view the 3D model
#📂: export the 3D model files